In [0]:
%pip install azure-eventhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/73.1 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/327.8 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 20.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
ehConf = {
    'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt("Endpoint=sb://iotcentral-streaming.servicebus.windows.net/;SharedAccessKeyName=iotcentral-streamjob_iotinput_policy;SharedAccessKey=fDbzj/BZkQnPcvzKm/1DYBN1pjVHG+LrX+AEhPc5Jkc=;EntityPath=smart-meter-stream"),
    'eventhubs.decodeUsingBase64': 'false'
}

event_hub_stream = (
    spark.readStream.format("eventhubs")
    .options(**ehConf)
    .load()
)

df = event_hub_stream.selectExpr("CAST(body AS STRING) as json_body")

schema = StructType([
    StructField("_eventcreationtime", StringType(), True),
    StructField("UsageKWh", DoubleType(), True),
    StructField("Voltage", DoubleType(), True),
    StructField("Current", DoubleType(), True),
    StructField("Location", StructType([
        StructField("alt", DoubleType(), True),
        StructField("lat", DoubleType(), True),
        StructField("lon", DoubleType(), True)
    ])),
    StructField("Timestamp", DoubleType(), True),
    StructField("_eventtype", StringType(), True),
    StructField("_timestamp", StringType(), True)
])

df_json = df.select(from_json(col("json_body"), schema).alias("data")).select("data.*")


In [0]:
# Save the parsed JSON DataFrame to a Bronze Delta table
df_json.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "/mnt/checkpoints/bronze") \
    .start("/mnt/datalake/bronze")
